# Projet Energie : préparation des données

1. Première exploration et description des données brutes issues de la source de données
    - quelques lignes exemples      
    - description
    - types
2. Data profiling : analyse de la qualité des données
3. Traitement des données. 
4. Ajout de données complémentaires

In [1]:
import pandas as pd
import numpy as np

In [2]:
csvPathCommun = '/content/eco2mix-regional-cons-def.csv'
csvPathlocal = 'eco2mix-regional-cons-def.csv'

df = pd.read_csv(csvPathlocal, sep=';')

## Analyse des données

In [3]:
df.head()

,Code INSEE région,Région,Nature,Date,Heure,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),...,TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Column 30
0,27,Bourgogne-Franche-Comté,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28,Normandie,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32,Hauts-de-France,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53,Bretagne,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,Île-de-France,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1683072 entries, 0 to 1683071
Data columns (total 32 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Code INSEE région    1683072 non-null  int64  
 1   Région               1683072 non-null  object 
 2   Nature               1683072 non-null  object 
 3   Date                 1683072 non-null  object 
 4   Heure                1683072 non-null  object 
 5   Date - Heure         1683072 non-null  object 
 6   Consommation (MW)    1683060 non-null  float64
 7   Thermique (MW)       1683060 non-null  float64
 8   Nucléaire (MW)       981785 non-null   float64
 9   Eolien (MW)          1682964 non-null  float64
 10  Solaire (MW)         1683060 non-null  float64
 11  Hydraulique (MW)     1683060 non-null  float64
 12  Pompage (MW)         946745 non-null   float64
 13  Bioénergies (MW)     1683060 non-null  float64
 14  Ech. physiques (MW)  1683060 non-null  float64
 15

## Préparation des données issues de la source principale

***Date et heure***

Création d'une colonne type Datetime

In [5]:
df['datetime'] = pd.to_datetime(df['Date - Heure'], format="%Y-%m-%dT%H:%M:%S%z", utc=True ).dt.tz_convert('Europe/Berlin').dt.tz_localize(None)

In [6]:
# Conversion du texte representant une date en objet type datetime afin de travailler plus efficacement à l'aide des methodes de datetime
# ne fonctionne plus ... df['datetime'] = pd.to_datetime(df['Date - Heure'], format="%Y-%m-%dT%H:%M:%S%z")
#df['datetime'] = df['datetime'].astype('datetime64[ns, UTC+01:00]')
df['datetime'].head()

0   2013-01-01
1   2013-01-01
2   2013-01-01
3   2013-01-01
4   2013-01-01
Name: datetime, dtype: datetime64[ns]

**Creation des colonnes indicatrices temporelles**

In [7]:
df['num_mois'] = df['datetime'].dt.month
df['annee'] = df['datetime'].dt.year
df['mois_sin'] = np.sin(df['datetime'].dt.month * 2 * np.pi / 12 )
df['mois_cos'] = np.cos(df['datetime'].dt.month * 2 * np.pi / 12 )
df['jour_sin'] = np.sin(df['datetime'].dt.day * 2 * np.pi / 31 )
df['jour_cos'] = np.cos(df['datetime'].dt.day * 2 * np.pi / 31 )
df['saison'] = df['num_mois'].\
replace(to_replace=[[1,2,3,12], [4,5],[6,7,8],[9,10,11]], value=[0, 1, 2, 1])

In [8]:
df = df.drop(columns=['Date','Heure','Date - Heure'])

***Colonne NATURE***

In [9]:
df.Nature.value_counts()

Nature
Données définitives    1683072
Name: count, dtype: int64

Colonne 'Nature' : On pourrait travailler que sur les données définitives c'est à dire jusqu'à fin 2020, et supprimer cette colonne. Ou alors simplement supprimer cette colonne en faisant confiance au données à partir de 2021.

In [10]:
df = df.drop(columns=['Nature'])

***Colonne 'column30'*** ne contient rien : supprimée

In [11]:
df['Column 30'].unique()

array([nan])

In [12]:
df = df.drop(columns=['Column 30'])

***QUALITE DES DONNEES***

***Valeurs manquantes***

In [13]:
# Characters such as empty strings '' or numpy.inf are not considered NA values (unless you set pd.options.mode.use_inf_as_na = True)
pd.options.mode.use_inf_as_na = True # pour qu'il considère une chaine de caractère vide comme une donnée manquante.
df.isna().sum()

Code INSEE région            0
Région                       0
Consommation (MW)           12
Thermique (MW)              12
Nucléaire (MW)          701287
Eolien (MW)                108
Solaire (MW)                12
Hydraulique (MW)            12
Pompage (MW)            736327
Bioénergies (MW)            12
Ech. physiques (MW)         12
Stockage batterie      1683072
Déstockage batterie    1683072
Eolien terrestre       1683072
Eolien offshore        1683072
TCO Thermique (%)      1472256
TCH Thermique (%)      1472256
TCO Nucléaire (%)      1560096
TCH Nucléaire (%)      1560096
TCO Eolien (%)         1472256
TCH Eolien (%)         1472256
TCO Solaire (%)        1472256
TCH Solaire (%)        1472256
TCO Hydraulique (%)    1472256
TCH Hydraulique (%)    1472256
TCO Bioénergies (%)    1472256
TCH Bioénergies (%)    1472256
datetime                     0
num_mois                     0
annee                        0
mois_sin                     0
mois_cos                     0
jour_sin

In [14]:
def alerts(df, thresh_na = 0.1):
    for col in df.columns:
        missingValuesRate = 100 * df[col].isna().sum()/len(df)
        if (missingValuesRate) > thresh_na:
            print('\nThe feature {} contains too many missing values({:.2f}%) Try to fix it !'\
                  .format(col,missingValuesRate))

alerts(df)


The feature Nucléaire (MW) contains too many missing values(41.67%) Try to fix it !

The feature Pompage (MW) contains too many missing values(43.75%) Try to fix it !

The feature Stockage batterie contains too many missing values(100.00%) Try to fix it !

The feature Déstockage batterie contains too many missing values(100.00%) Try to fix it !

The feature Eolien terrestre contains too many missing values(100.00%) Try to fix it !

The feature Eolien offshore contains too many missing values(100.00%) Try to fix it !

The feature TCO Thermique (%) contains too many missing values(87.47%) Try to fix it !

The feature TCH Thermique (%) contains too many missing values(87.47%) Try to fix it !

The feature TCO Nucléaire (%) contains too many missing values(92.69%) Try to fix it !

The feature TCH Nucléaire (%) contains too many missing values(92.69%) Try to fix it !

The feature TCO Eolien (%) contains too many missing values(87.47%) Try to fix it !

The feature TCH Eolien (%) contains too

***Analyse des valeurs manquantes sur les colonnes TCO XXX et TCH XXX***

In [15]:
# select columns by regular expression
columns_TC = df.filter(regex='^TC.*', axis=1).columns

In [16]:
for tc_colonne in columns_TC:
    print(tc_colonne, ', années avec données manquantes : ', df[df[tc_colonne].isna()]['datetime'].dt.year.unique())

TCO Thermique (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019]
TCH Thermique (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019]
TCO Nucléaire (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019 2020]
TCH Nucléaire (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019 2020]
TCO Eolien (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019]
TCH Eolien (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019]
TCO Solaire (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019]
TCH Solaire (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019]
TCO Hydraulique (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019]
TCH Hydraulique (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018 2019]
TCO Bioénergies (%) , années avec données manquantes :  [2013 2014 2015 2016 2017 2018

In [17]:
alerts( df[df['datetime'].dt.year == 2020] )


The feature Nucléaire (MW) contains too many missing values(41.67%) Try to fix it !

The feature Pompage (MW) contains too many missing values(41.67%) Try to fix it !

The feature Stockage batterie contains too many missing values(100.00%) Try to fix it !

The feature Déstockage batterie contains too many missing values(100.00%) Try to fix it !

The feature Eolien terrestre contains too many missing values(100.00%) Try to fix it !

The feature Eolien offshore contains too many missing values(100.00%) Try to fix it !

The feature TCO Nucléaire (%) contains too many missing values(41.67%) Try to fix it !

The feature TCH Nucléaire (%) contains too many missing values(41.67%) Try to fix it !


***Suppression des colonnes sans aucune données de production***

In [18]:
df = df.drop(columns=['Stockage batterie', 'Déstockage batterie',\
                      'Eolien terrestre','Eolien offshore'])

***Quelles sont les lignes où la consommation n'est pas renseignée ?***

In [19]:
df[df['Consommation (MW)'].isna()].datetime

0    2013-01-01
1    2013-01-01
2    2013-01-01
3    2013-01-01
4    2013-01-01
5    2013-01-01
6    2013-01-01
7    2013-01-01
8    2013-01-01
9    2013-01-01
10   2013-01-01
11   2013-01-01
Name: datetime, dtype: datetime64[ns]

Suppression des 12 lignes (1er janvier 2013 à minuit) où la valeurs de consommation est absente.

In [20]:
df = df.drop(index=df[df['Consommation (MW)'].isna()].index)

***Quelles sont les lignes où la filière nucleaire est renseignée ?***

In [21]:
df[df['Nucléaire (MW)'].notna()].datetime.dt.year.unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020], dtype=int32)

***Quelles sont les lignes où la filière pompage est renseignée ?***

In [22]:
df[df['Pompage (MW)'].notna()].datetime.dt.year.unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020], dtype=int32)

In [23]:
df['Pompage (MW)'].unique()

array([   nan,     0.,   -13., ..., -2420., -2054., -2298.])

Remarque : la filière 'Pompage' est de l'énergie soustraite à la production

***Vérifications de la présence de doublons***

In [24]:
df_2013 = (df[(df.datetime.dt.year == 2013) & (df['Code INSEE région']==24)]).copy()
df_2013.loc[df.duplicated(subset=['Code INSEE région', 'datetime'])]

,Code INSEE région,Région,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),...,TCO Bioénergies (%),TCH Bioénergies (%),datetime,num_mois,annee,mois_sin,mois_cos,jour_sin,jour_cos,saison
51321,24,Centre-Val de Loire,2737.0,85.0,8613.0,159.0,0.0,52.0,NaN,25.0,...,NaN,NaN,2013-03-31 03:00:00,3,2013,1.0,6.123234e-17,-2.449294e-16,1.0,3
51355,24,Centre-Val de Loire,2831.0,86.0,8593.0,156.0,0.0,51.0,NaN,28.0,...,NaN,NaN,2013-03-31 03:30:00,3,2013,1.0,6.123234e-17,-2.449294e-16,1.0,3


Bizzare : il considère un doublon là où il y a 2 valeurs qui pourtant n'ont pas le meme datetime.

***Création des colonnes 'production_brute' et 'production_nette'***
'production_brute' : Il s'agit de la production totale nationale
'production_nette' : Il s'agit de la production brute auquel a été rajouté les échanges avec pays et les données 'Pompage' (de l'énergie consommée).   
**Remarque :** les valeurs s'expriment en MW ; il s'agit donc d'une puissance sur le pas de temps indiqué ici 0,5h. Il faut donc multiplier par ce temps en heure pour avoir la valeur d'énergie en MWh.

In [25]:
df.columns

Index(['Code INSEE région', 'Région', 'Consommation (MW)', 'Thermique (MW)',
       'Nucléaire (MW)', 'Eolien (MW)', 'Solaire (MW)', 'Hydraulique (MW)',
       'Pompage (MW)', 'Bioénergies (MW)', 'Ech. physiques (MW)',
       'TCO Thermique (%)', 'TCH Thermique (%)', 'TCO Nucléaire (%)',
       'TCH Nucléaire (%)', 'TCO Eolien (%)', 'TCH Eolien (%)',
       'TCO Solaire (%)', 'TCH Solaire (%)', 'TCO Hydraulique (%)',
       'TCH Hydraulique (%)', 'TCO Bioénergies (%)', 'TCH Bioénergies (%)',
       'datetime', 'num_mois', 'annee', 'mois_sin', 'mois_cos', 'jour_sin',
       'jour_cos', 'saison'],
      dtype='object')

In [26]:
def sum_prod(row, *prod_columns):
    total_prod = 0.0
    for prd_colon in prod_columns:
        if (pd.notna(row[prd_colon])) :
            total_prod += row[prd_colon]
    return total_prod

In [27]:
prod_nette_columns = ('Thermique (MW)','Nucléaire (MW)','Eolien (MW)','Solaire (MW)','Hydraulique (MW)','Bioénergies (MW)')
df['production_nette'] = df.apply(sum_prod , args=prod_nette_columns, axis=1)

In [ ]:
prod_brute_columns = ['Thermique (MW)','Nucléaire (MW)', 'Eolien (MW)',\
                 'Solaire (MW)', 'Hydraulique (MW)', 'Pompage (MW)',\
                 'Bioénergies (MW)', 'Ech. physiques (MW)']
df['production_brute'] = df.apply(sum_prod , args=(prod_brute_columns), axis=1)

In [ ]:
# Renommage en minuscule, sans espaces
df.rename(columns={'Code INSEE région':'code_region', 'Région':'region', 'Consommation (MW)':'consommation', 'Thermique (MW)':'thermique','Nucléaire (MW)':'nucleaire', 'Eolien (MW)':'eolien', 'Solaire (MW)':'solaire', 'Hydraulique (MW)':'hydraulique','Pompage (MW)':'pompage', 'Bioénergies (MW)':'bioenergies', 'Ech. physiques (MW)':'ech.physiques','TCO Thermique (%)':'tco_thermique', 'TCH Thermique (%)':'tch_thermique', 'TCO Nucléaire (%)':'tco_nucléaire','TCH Nucléaire (%)':'tch_nucleaire', 'TCO Eolien (%)':'tco_eolien', 'TCH Eolien (%)':'tch_eolien','TCO Solaire (%)':'tco_solaire', 'TCH Solaire (%)':'tch_solaire', 'TCO Hydraulique (%)':'tco_hydraulique','TCH Hydraulique (%)':'tch_hydraulique', 'TCO Bioénergies (%)':'tco_bioenergies', 'TCH Bioénergies (%)':'tch_bioenergies'}, inplace=True)

***Colonnes définitives présentes désormais dans le Dataframe 'df'***

In [ ]:
pd.DataFrame(data={'colonne':df.columns, 'type':df.dtypes.values}, index=range( 1, (len(df.columns) + 1) ) )

On obtient à la fin de ce Notebook un df avec :

- les colonnes sans données utiles supprimées
- les lignes avec absences de consommations retirées

- une nouvelle colonne 'datetime' type datetime64 qui exprime precisement le temps et avec laquelle on peut travailler avec n'importe quel pas de temps (regroupement, aggrégation par exemple)
- une nouvelle colonne 'num_mois' un nombre entier précisant le mois de l'année (utile pour accès direct à la saison)
- les colonnes 'production_brute' et 'production_nette'  
- 'production_brute' : Il s'agit de la production totale nationale   
- 'production_nette' : Ils s'agit de la production brute auquel a été rajouté les échanges avec pays et les données 'Pompage' (de l'énergie consommée).   
   
**Remarque :** les valeurs s'expriment en MW ; il s'agit donc d'une puissance sur le pas de temps indiqué ici 0,5h. Il faut donc multiplier par ce temps en heure pour avoir la valeur d'énergie en MWh.
Remarque : Il faudra peut-être renommer les colonnes dont le nom contient des espaces : si ca pose des problème.

In [ ]:
#Creation d'un fichier csv contenant le dataframe préparé
df.to_csv("eco2mix-regional-prepare.csv", sep=";")

# Groupement des données par Jour et par région

In [ ]:
df_jour = df.groupby(by=['code_region',pd.Grouper(key='datetime', freq='D')]).agg({'region': lambda x : x.iloc[0] , 'annee': lambda x : x.iloc[0] , 'num_mois': lambda x : x.iloc[0] ,\
'saison': lambda x : x.iloc[0], 'mois_sin' : lambda x : x.iloc[0], 'mois_cos': lambda x : x.iloc[0], 'jour_sin': lambda x : x.iloc[0], 'jour_cos': lambda x : x.iloc[0],\
'consommation': sum, 'thermique':sum, 'nucleaire':sum,'eolien':sum, 'solaire':sum, 'hydraulique':sum, 'pompage':sum, 'bioenergies':sum,'ech.physiques':sum,\
'tco_thermique': np.mean , 'tch_thermique': np.mean , 'tco_nucléaire': np.mean ,'tch_nucleaire': np.mean , 'tco_eolien': np.mean , 'tch_eolien': np.mean , \
 'tco_solaire': np.mean ,'tch_solaire': np.mean , 'tco_hydraulique': np.mean ,\
'tch_hydraulique': np.mean , 'tco_bioenergies': np.mean ,'tch_bioenergies': np.mean, 'production_brute' : sum, 'production_nette' : sum })
df_jour = df_jour.reset_index()
df_jour.head()

Ajout des températures quotidiennes régionales
------------------------------------------------

In [ ]:
df_temperatures = pd.read_csv("temperature-quotidienne-regionale.csv", parse_dates=['Date'],  sep=";" )

In [ ]:
df_temperatures.drop(columns=['Région', 'ID'], inplace=True)
df_temperatures.rename(columns={'Date':'date', 'Code INSEE région': 'code_region', 'TMin (°C)': 'TMin','TMax (°C)':'TMax','TMoy (°C)':'TMoy'}, inplace=True)

In [ ]:
df_jour_temp = df_jour.merge(right=df_temperatures, how='left',left_on=['datetime','code_region'], right_on=['date','code_region']).drop(columns=['date'])

In [ ]:
# on remarque que l'on a des temperatures qu'à partir de l'année 2016
df_jour_temp[df_jour_temp.TMin.notna()]['datetime']

In [ ]:
#Creation d'un fichier csv contenant le dataframe préparé
df_jour_temp.to_csv("eco2mix-prepare-temperatures.csv", sep=";")

Ajout des données liées aux pannes de production
--------------

In [ ]:
indispo = pd.read_csv('defaut_production_moy_jour.csv' , parse_dates=['datetime'], index_col=0)
indispo.reset_index(drop=True, inplace=True)

In [ ]:
indispo.head()

In [ ]:
indispo.dtypes

In [ ]:
# afin de ne rajouter qu'une seule fois les données d'energie manquante, *
# on choisi une région au hasard afin de lui associer la valeur d'energie manquante nationale

codeRegion11Serie = pd.DataFrame({"code_region" : np.ones((len(indispo)),dtype='int')*11 })
indispo_ileDeFrance = pd.concat([indispo,codeRegion11Serie], axis=1)

In [ ]:
df_jour_temp_indispo = df_jour_temp.merge(right=indispo_ileDeFrance, how='left',left_on=['datetime', 'code_region'], right_on=['datetime','code_region'])

In [ ]:
df_jour_temp_indispo[df_jour_temp_indispo['defaut_energie_moy_jour'].notna()]

In [ ]:
#Creation d'un fichier csv contenant le dataframe préparé
df_jour_temp_indispo.to_csv("eco2mix-prepare-temperatures_indispo.csv", sep=";")

Ajout des données liées aux prix de contrats de base et contrats heures creuses - heures pleines
-------------------------------

In [ ]:
# Premier jeu de données : contrat de base
df = pd.read_csv("TRV électricité inf36 BASE_2012- S2 2023.csv", sep=";")

In [ ]:
# Création d'une colonne de prix total

df["PART_FIXE_TTC"] = df["PART_FIXE_TTC"].apply(lambda x: x.replace(',','.').replace(',','.')).astype("float")

df["PART_VARIABLE_TTC"] = df["PART_VARIABLE_TTC"].apply(lambda x : x.replace(',','.').replace(',','.')).astype("float")

df["prix_base_moyen_ttc"] = df["PART_FIXE_TTC"] + df ["PART_VARIABLE_TTC"]

In [ ]:
# Regroupement par tarif par date et calcul du prix de base moyen indépendamment de la puissance souscrite
df_base = df.groupby(["DATE_DEBUT","DATE_FIN"], as_index = False).agg({"prix_base_moyen_ttc" : "mean"})

df_base["annee"]= df_base["DATE_FIN"].str[6:].astype("int")

df_base["mois"] = df_base["DATE_FIN"].str[3:5].astype("int")

In [ ]:
df_base["DATE_DEBUT"] = pd.to_datetime(df_base["DATE_DEBUT"], format='%d/%m/%Y')

df_base["DATE_FIN"] = pd.to_datetime(df_base["DATE_FIN"], format='%d/%m/%Y')

In [ ]:
# Création d'un tarif par jour
def getDatetimesArray(row):
    return pd.date_range(start=row["DATE_DEBUT"],
                         end=row['DATE_FIN'],freq='D').values

df_base['date'] = df_base.apply(getDatetimesArray, axis=1)

df_base = df_base.explode('date')

df_base.shape

In [ ]:
df_base.sort_values(by="date").head()

In [ ]:
# Second jeu de données : Contrat heures pleines - heures creuses
df2 = pd.read_csv("TRV électricité inf36 HPHC_2012- S2 2023.csv", sep = ";")


In [ ]:
df2.tail()

In [ ]:
df2.info()

In [ ]:
# df2 # Création d'une colonne de prix total

df2["PART_FIXE_TTC"] = df2["PART_FIXE_TTC"].apply(lambda x: x.replace(',','.').replace(',','.')).astype("float")

df2["PART_VARIABLE_HC_TTC"] = df2["PART_VARIABLE_HC_TTC"].apply(lambda x : x.replace(',','.').replace(',','.')).astype("float")

df2["PART_VARIABLE_HP_TTC"] = df2["PART_VARIABLE_HP_TTC"].apply(lambda x : x.replace(',','.').replace(',','.')).astype("float")

df2["prix_HC_ttc"] = df2["PART_FIXE_TTC"] + df2["PART_VARIABLE_HC_TTC"]

df2["prix_HP_ttc"] = df2["PART_FIXE_TTC"] + df2["PART_VARIABLE_HP_TTC"]

df2["prix_HC_HP_moy_ttc"] = (df2["prix_HC_ttc"] + df2["prix_HP_ttc"])/2

df2.head()

In [ ]:
# Regroupement par tarif par date et calcul du prix hc-hp moyen indépendamment de la puissance souscrite
df2_hc_hp = df2.groupby(["DATE_DEBUT","DATE_FIN"], as_index = False).agg({"prix_HC_HP_moy_ttc" : "mean"})

df2_hc_hp["annee"]= df2_hc_hp["DATE_FIN"].str[6:].astype("int")

df2_hc_hp["mois"] = df2_hc_hp["DATE_FIN"].str[3:5].astype("int")

In [ ]:
df2_hc_hp.sort_values(["annee"], ascending=[True]).head()

In [ ]:
df2_hc_hp["DATE_DEBUT"] = pd.to_datetime(df2_hc_hp["DATE_DEBUT"], format='%d/%m/%Y')

df2_hc_hp["DATE_FIN"] = pd.to_datetime(df2_hc_hp["DATE_FIN"], format='%d/%m/%Y')

In [ ]:
df2_hc_hp = df2_hc_hp.dropna(subset=["DATE_FIN"])

In [ ]:
# Création d'un tarif par jour
df2_hc_hp['date'] = df2_hc_hp.apply(getDatetimesArray, axis=1)

In [ ]:
df2_hc_hp = df2_hc_hp.explode('date')

df2_hc_hp.shape

In [ ]:
df2_hc_hp.sort_values(by="date").head()

In [ ]:
# Création d'un dataframe fusionné avec les deux colonnes de prix
df_prix = df_base.merge(df2_hc_hp, how = "left", on="date")

In [ ]:
cols = ["DATE_DEBUT_x","DATE_FIN_x","annee_x","mois_x","DATE_DEBUT_y","DATE_FIN_y","annee_y","mois_y"]

df_prix = df_prix.drop(columns = cols)

In [ ]:
df_prix = df_prix[df_prix["date"]> "2012-12-31"]

In [ ]:
df_prix = df_prix[df_prix["date"]< "2021-01-01"]

In [ ]:
df_prix.sort_values(["date"], ascending=[False]).head()

In [ ]:
# Fusion avec df consolidé
df_jour_temp_indispo_prix = df_jour_temp_indispo.merge(right=df_prix, how='left',left_on=['datetime'], right_on=['date'])
df_jour_temp_indispo_prix = df_jour_temp_indispo_prix.drop("date", axis=1)
df_jour_temp_indispo_prix

In [ ]:
df_jour_temp_indispo_prix[df_jour_temp_indispo_prix.duplicated()]

Ajout de l'indication 'jour_off' si le jour est férié ou dimanche
-----------------------

In [ ]:
jf_df = pd.read_csv("jours_feries_metropole.csv", parse_dates=['date'])
jf_list = jf_df.date.dt.strftime('%Y-%m-%d').to_list()

In [ ]:
df_jour_temp_indispo_prix['jour_off'] = df_jour_temp_indispo_prix['datetime'].dt.weekday == 6 |  df_jour_temp_indispo_prix['datetime'].dt.strftime('%Y-%m-%d').isin(jf_list)

In [ ]:
df_jour_temp_indispo_prix[df_jour_temp_indispo_prix['jour_off']]

In [ ]:
df_jour_temp_indispo_prix[df_jour_temp_indispo_prix.duplicated()]

## Dataframe définitif pour les modélisations

In [ ]:
#Creation d'un fichier csv contenant le dataframe préparé
df_jour_temp_indispo_prix.to_csv("eco2mix-prepare-temperatures_indispo_prix.csv", sep=";")

In [ ]:
df = df_jour_temp_indispo_prix
df.info()

## Modélisation Random Forest, GradientBoostingRegressor

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

X = df.drop('consommation', axis=1)
X = X.drop('datetime', axis=1)
X = X.drop('region', axis=1)
X = X.drop('code_region', axis=1)
y = df['consommation']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

col = ['tco_thermique','tch_thermique','tco_nucléaire','tch_nucleaire','tco_eolien','tch_eolien','tco_solaire',\
        'tch_solaire','tco_hydraulique','tch_hydraulique','tco_bioenergies','tch_bioenergies','TMin','TMax','TMoy']
col_train = X_train[col]
col_test = X_test[col]

# Remplacement des NANs des colonnes tco - tch et température par leurs médianes respectives
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
X_train.loc[:,col] = imputer.fit_transform(col_train)
X_test.loc[:,col] = imputer.transform(col_test)


In [ ]:
#Remplacement des NaN par 0 dans la colonne “defaut énergie moy jour”
X_train.defaut_energie_moy_jour = X_train.defaut_energie_moy_jour.fillna(0)
X_test.defaut_energie_moy_jour = X_test.defaut_energie_moy_jour.fillna(0)

# Standardisation des variables numériques
col_num_train = X_train.iloc[:,1:]
col_num_test = X_test.iloc[:,1:]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
col_num_train = scaler.fit_transform(col_num_train)
col_num_test = scaler.fit(col_num_test)

In [ ]:
rf=RandomForestRegressor()
rf.fit(X_train,y_train)
print(rf.score(X_train,y_train))

In [ ]:
test_score = rf.score(X_test, y_test)
print(f"Score sur l'ensemble de test : {test_score}")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 4))
plt.scatter(y_train, rf.predict(X_train), alpha=1)
plt.title('Graphique de dispersion des vraies valeurs par rapport aux prédictions')
plt.xlabel('Valeurs réelles')
plt.ylabel('Prédictions')
plt.show()

In [ ]:
feature_importances = rf.feature_importances_

importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

#Graphique
plt.figure(figsize=(8, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Fonctionnalités')
plt.title('Importance des fonctionnalités dans le modèle Random Forest')
plt.show()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

feature_names = X_train[col].columns

# Création et entraînement du modèle Gradient Boosting
gb = GradientBoostingRegressor()
gb.fit(X_train[col], y_train)

predictions_train_gb = gb.predict(X_train[col])

r2_train_gb = gb.score(X_train[col], y_train)
print(f"R² sur les données d'entraînement (Gradient Boosting): {r2_train_gb}")

importances_gb = gb.feature_importances_
indices_gb = np.argsort(importances_gb)[::-1]

plt.figure(figsize=(11, 5))
plt.bar(range(len(importances_gb)), importances_gb[indices_gb], align='center')
plt.xticks(range(len(importances_gb)), feature_names[indices_gb], rotation=45)
plt.title('Importance des caractéristiques (Gradient Boosting)')
plt.show()

In [ ]:
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"Mean Squared Error sur l'ensemble de test : {mse}")

In [ ]:
# Nouveau modèle

In [ ]:
# Diviser les données en variables explicatives (X) et la variable cible (y)
X = df.drop('consommation', axis=1)
X =X.drop('datetime', axis=1)
X =X.drop('region', axis=1)
X = X.drop('code_region', axis=1)
y = df['consommation']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

col = ['saison','mois_sin','mois_cos','jour_sin','jour_cos','defaut_energie_moy_jour','prix_base_moyen_ttc',\
        'prix_HC_HP_moy_ttc','TMin','TMax','TMoy']
col_train = X_train[col]
col_test = X_test[col]

# Remplacement des NANs des colonnes tco - tch et température par leurs médianes respectives
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
X_train.loc[:,col] = imputer.fit_transform(col_train)
X_test.loc[:,col] = imputer.transform(col_test)

In [ ]:
# Standardisation des variables numériques
col_num_train = X_train.iloc[:,1:]
col_num_test = X_test.iloc[:,1:]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
col_num_train = scaler.fit_transform(col_num_train)
col_num_test = scaler.fit(col_num_test)

In [ ]:
rf=RandomForestRegressor()
rf.fit(X_train[col],y_train)
print(rf.score(X_train[col],y_train))

In [ ]:
y_pred_rf = rf.predict(X_train[col])

# Création d'un scatter plot avec Matplotlib
plt.scatter(y_train, y_pred_rf, alpha=0.5)
plt.xlabel('Vraies Valeurs')
plt.ylabel('Prédictions')
plt.title('Graphique de dispersion des vraies valeurs par rapport aux prédictions (Random Forest)')
plt.show()

In [ ]:
importances = rf.feature_importances_
feature_names = X_train[col].columns

# Tri des indices par ordre décroissant d'importance
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 5))
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), feature_names[indices], rotation=45)
plt.title('Importance des caractéristiques')
plt.show()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Création et entraînement du modèle Gradient Boosting
gb = GradientBoostingRegressor()
gb.fit(X_train[col], y_train)

# Prédictions sur les données d'entraînement
predictions_train_gb = gb.predict(X_train[col])

r2_train_gb = gb.score(X_train[col], y_train)
print(f"R² sur les données d'entraînement (Gradient Boosting): {r2_train_gb}")

importances_gb = gb.feature_importances_
indices_gb = np.argsort(importances_gb)[::-1]

plt.figure(figsize=(11, 5))
plt.bar(range(len(importances_gb)), importances_gb[indices_gb], align='center')
plt.xticks(range(len(importances_gb)), feature_names[indices_gb], rotation=45)
plt.title('Importance des caractéristiques (Gradient Boosting)')
plt.show()

In [ ]:
predictions_test_gb = gb.predict(X_test[col])

mse_test_gb = mean_squared_error(y_test, predictions_test_gb)

print(f"Mean Squared Error sur l'ensemble de test (Gradient Boosting): {mse_test_gb}")

In [ ]:
# Nouveau modèle

In [ ]:
df = df_jour_temp_indispo_prix

In [ ]:
# Diviser les données en variables explicatives (X) et la variable cible (y)
X = df.drop('consommation', axis=1)
X =X.drop('datetime', axis=1)
X =X.drop('region', axis=1)
X = X.drop('code_region', axis=1)
y = df['consommation']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

col = ['saison','defaut_energie_moy_jour','prix_base_moyen_ttc',\
        'prix_HC_HP_moy_ttc','TMin','TMax','TMoy']
col_train = X_train[col]
col_test = X_test[col]

# Remplacement des NANs des colonnes tco - tch et température par leurs médianes respectives
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
X_train.loc[:,col] = imputer.fit_transform(col_train)
X_test.loc[:,col] = imputer.transform(col_test)

In [ ]:
# Remplacement des NANs par 0
X_train.loc[:, col] = col_train.fillna(0)
X_test.loc[:, col] = col_test.fillna(0)

# Standardisation des variables numériques
col_num_train = X_train.iloc[:,1:]
col_num_test = X_test.iloc[:,1:]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
col_num_train = scaler.fit_transform(col_num_train)
col_num_test = scaler.fit(col_num_test)

In [ ]:
rf=RandomForestRegressor()
rf.fit(X_train[col],y_train)
print(rf.score(X_train[col],y_train))

In [ ]:
y_pred_rf = rf.predict(X_train[col])
result_rf_df = pd.DataFrame({'Vraies Valeurs': y_train, 'Prédictions': y_pred_rf})

plt.scatter(result_rf_df['Vraies Valeurs'], result_rf_df['Prédictions'], alpha=0.5)
plt.xlabel('Vraies Valeurs')
plt.ylabel('Prédictions')
plt.title('Graphique de dispersion des vraies valeurs par rapport aux prédictions')
plt.show()

In [ ]:
importances = rf.feature_importances_
feature_names = X_train[col].columns

# Tri des indices par ordre décroissant d'importance
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 5))
plt.bar(range(len(importances)), importances[indices], align='center')
plt.xticks(range(len(importances)), feature_names[indices], rotation=45)
plt.title('Importance des caractéristiques')
plt.show()

In [ ]:
# Création et entraînement du modèle Gradient Boosting
gb = GradientBoostingRegressor()
gb.fit(X_train[col], y_train)

# Prédictions sur les données d'entraînement
predictions_train_gb = gb.predict(X_train[col])

r2_train_gb = gb.score(X_train[col], y_train)
print(f"R² sur les données d'entraînement (Gradient Boosting): {r2_train_gb}")

importances_gb = gb.feature_importances_
indices_gb = np.argsort(importances_gb)[::-1]

plt.figure(figsize=(11, 5))
plt.bar(range(len(importances_gb)), importances_gb[indices_gb], align='center')
plt.xticks(range(len(importances_gb)), feature_names[indices_gb], rotation=45)
plt.title('Importance des caractéristiques (Gradient Boosting)')
plt.show()

In [ ]:
predictions_test_gb = gb.predict(X_test[col])

mse_test_gb = mean_squared_error(y_test, predictions_test_gb)

print(f"Mean Squared Error sur l'ensemble de test (Gradient Boosting): {mse_test_gb}")

## Modélisations GradientBoostingRegressor, Metrics, Ridge et Lasso

In [ ]:
dfe = df_jour_temp_indispo_prix

In [ ]:
dfd = dfe[dfe.duplicated()]

In [ ]:
#Identification des NaN
dfe[~dfe["defaut_energie_moy_jour"].isna()].head()

In [ ]:
# Exploration des modalités des variables catégorielles

print(dfe[["region","code_region"]].value_counts())

In [ ]:
# L'information entre le code région et la région est redondante, je supprime code région du dfe
dfe = dfe.drop("code_region", axis=1)

In [ ]:
# Identification de la variable cible et des variables explicatives
target = dfe["consommation"]

feats = dfe.drop("consommation", axis =1)

In [ ]:
#Séparation du jeu de données en jeu d'entraînement et jeu de test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size = 0.2)


In [ ]:
col = ["tco_thermique","tch_thermique","tco_nucléaire","tch_nucleaire","tco_eolien","tch_eolien","tco_solaire",\
        "tch_solaire","tco_hydraulique","tch_hydraulique","tco_bioenergies","tch_bioenergies","TMin","TMax","TMoy"]

col_train = X_train[col]

col_test = X_test[col]

In [ ]:
# Remplacement des NANs des colonnes tco - tch et température par leurs médianes respectives
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = "median")

X_train.loc[:,col] = imputer.fit_transform(col_train)

X_test.loc[:,col] = imputer.transform(col_test)

In [ ]:
#Remplacement des NaN par 0 dans la colonne "defaut énergie moy jour"

X_train.defaut_energie_moy_jour = X_train.defaut_energie_moy_jour.fillna(0)

X_test.defaut_energie_moy_jour = X_test.defaut_energie_moy_jour.fillna(0)

In [ ]:
# Standardisation des variables numériques soit colonnes 3 à 30

col_num_train = X_train.iloc[:,2:]

col_num_test = X_test.iloc[:,2:]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

col_num_train = scaler.fit_transform(col_num_train)

col_num_test = scaler.fit(col_num_test)

In [ ]:
# Encodage de la variable explicative region - pandas.get_dummies

X_train = pd.get_dummies(X_train)

X_test = pd.get_dummies(X_test)

In [ ]:
# Encodage de la variable datetime
import datetime as dt
X_train['datetime']=X_train['datetime'].map(dt.datetime.toordinal)

X_test['datetime']=X_test['datetime'].map(dt.datetime.toordinal)

In [ ]:
# Première modélisation : GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(random_state = 0)

reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
print("Score train :",reg.score(X_train,y_train))

print("Score test :",reg.score(X_test,y_test))

In [ ]:
# Afficher représentation de pred test

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10))

plt.scatter(y_pred, y_test, c="green")

plt.plot((y_test.min(),y_test.max()),(y_test.min(),y_test.max()), color = "red")

plt.xlabel("Prédiction")
plt.ylabel("Vraie valeur")

plt.title("Gradient Boosting Regressor pour la prédiction de la consommation")

plt.show();

In [ ]:
# Afficher les feature importances

print(reg.feature_importances_)

feat_imp = pd.DataFrame(reg.feature_importances_, index = X_train.columns, columns = ["Importance"])

feat_imp.sort_values(by = "Importance", ascending = False)

feat_imp.plot(kind = "bar", figsize=(8,8))

plt.show();

In [ ]:
# Seconde modélisation : Suppression de la variable production nette

dfe = dfe.drop("production_nette", axis = 1)

target = dfe["consommation"]

feats = dfe.drop("consommation", axis =1)

#Séparation du jeu de données en jeu d'entraînement et jeu de test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size = 0.2)

col = ["tco_thermique","tch_thermique","tco_nucléaire","tch_nucleaire","tco_eolien","tch_eolien","tco_solaire",\
        "tch_solaire","tco_hydraulique","tch_hydraulique","tco_bioenergies","tch_bioenergies","TMin","TMax","TMoy"]

col_train = X_train[col]

col_test = X_test[col]

# Remplacement des NANs des colonnes tco - tch et température par leurs médianes respectives
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = "median")

X_train.loc[:,col] = imputer.fit_transform(col_train)

X_test.loc[:,col] = imputer.transform(col_test)

#Remplacement des NaN par 0 dans la colonne "defaut énergie moy jour"

X_train.defaut_energie_moy_jour = X_train.defaut_energie_moy_jour.fillna(0)

X_test.defaut_energie_moy_jour = X_test.defaut_energie_moy_jour.fillna(0)

# Standardisation des variables numériques soit colonnes 3 à 30

col_num_train = X_train.iloc[:,2:]

col_num_test = X_test.iloc[:,2:]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

col_num_train = scaler.fit_transform(col_num_train)

col_num_test = scaler.fit(col_num_test)

# Encodage de la variable explicative region - pandas.get_dummies

X_train = pd.get_dummies(X_train)

X_test = pd.get_dummies(X_test)

# Encodage de la variable datetime
import datetime as dt
X_train['datetime']=X_train['datetime'].map(dt.datetime.toordinal)

X_test['datetime']=X_test['datetime'].map(dt.datetime.toordinal)

In [ ]:
# Seconde modélisation : GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(random_state = 0)

reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
y_pred = reg.predict(X_test)

print("Score train :",reg.score(X_train,y_train))

print("Score test :",reg.score(X_test,y_test))

In [ ]:
# Afficher représentation de pred test

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10))

plt.scatter(y_pred, y_test, c="green")

plt.plot((y_test.min(),y_test.max()),(y_test.min(),y_test.max()), color = "red")

plt.xlabel("Prédiction")
plt.ylabel("Vraie valeur")

plt.title("Gradient Boosting Regressor pour la prédiction de la consommation")

plt.show();

In [ ]:
# Afficher les feature importances

print(reg.feature_importances_)

feat_imp = pd.DataFrame(reg.feature_importances_, index = X_train.columns, columns = ["Importance"])

feat_imp.sort_values(by = "Importance", ascending = False)

feat_imp.plot(kind = "bar", figsize=(8,8))

plt.show();

In [ ]:
# Troisième modélisation : Suppression des variables explicatives liées à la production 

target = dfe["consommation"]

feats = dfe[["datetime","annee","num_mois","saison","mois_sin","mois_cos","jour_sin","jour_cos","TMin","TMax","TMoy",
             "defaut_energie_moy_jour","prix_base_moyen_ttc","prix_HC_HP_moy_ttc","jour_off"]]

#Séparation du jeu de données en jeu d'entraînement et jeu de test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size = 0.2)

col = ["TMin","TMax","TMoy"]

col_train = X_train[col]

col_test = X_test[col]

# Remplacement des NANs des colonnes température par leurs médianes respectives
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = "median")

X_train.loc[:,col] = imputer.fit_transform(col_train)

X_test.loc[:,col] = imputer.transform(col_test)

#Remplacement des NaN par 0 dans la colonne "defaut énergie moy jour"

X_train.defaut_energie_moy_jour = X_train.defaut_energie_moy_jour.fillna(0)

X_test.defaut_energie_moy_jour = X_test.defaut_energie_moy_jour.fillna(0)

X_train.info()

In [ ]:
# Standardisation des variables numériques 

col_num_train = X_train.iloc[:,1:14]

col_num_test = X_test.iloc[:,1:14]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

col_num_train = scaler.fit_transform(col_num_train)

col_num_test = scaler.fit(col_num_test)

# Encodage de la variable datetime
import datetime as dt
X_train['datetime']=X_train['datetime'].map(dt.datetime.toordinal)

X_test['datetime']=X_test['datetime'].map(dt.datetime.toordinal)


# Troisième modélisation : GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(random_state = 0)

reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
y_pred = reg.predict(X_test)

print("Score train :",reg.score(X_train,y_train))

print("Score test :",reg.score(X_test,y_test))

In [ ]:
# Afficher représentation de pred test

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10))

plt.scatter(y_pred, y_test, c="green")

plt.plot((y_test.min(),y_test.max()),(y_test.min(),y_test.max()), color = "red")

plt.xlabel("Prédiction")
plt.ylabel("Vraie valeur")

plt.title("Gradient Boosting Regressor pour la prédiction de la consommation")

plt.show();

In [ ]:
# Afficher les feature importances

print(reg.feature_importances_)

feat_imp = pd.DataFrame(reg.feature_importances_, index = X_train.columns, columns = ["Importance"])

feat_imp.sort_values(by = "Importance", ascending = False)

feat_imp.plot(kind = "bar", figsize=(8,8))

plt.show();

In [ ]:
# # Dernière modélisation avec les variables explicatives : defaut_energie_moy_jour, mois_cos, TMoy, TMax

target = dfe["consommation"]

feats = dfe[["defaut_energie_moy_jour","mois_cos","TMax","TMoy"]]

#Séparation du jeu de données en jeu d'entraînement et jeu de test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size = 0.2)

col = ["TMax","TMoy"]

col_train = X_train[col]

col_test = X_test[col]

# Remplacement des NANs des colonnes température par leurs médianes respectives
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = "median")

X_train.loc[:,col] = imputer.fit_transform(col_train)

X_test.loc[:,col] = imputer.transform(col_test)

#Remplacement des NaN par 0 dans la colonne "defaut énergie moy jour"

X_train.defaut_energie_moy_jour = X_train.defaut_energie_moy_jour.fillna(0)

X_test.defaut_energie_moy_jour = X_test.defaut_energie_moy_jour.fillna(0)


# Standardisation des variables numériques soit colonnes 1 à 3

col_num_train = X_train.iloc[:,1:]

col_num_test = X_test.iloc[:,1:]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

col_num_train = scaler.fit_transform(col_num_train)

col_num_test = scaler.fit(col_num_test)


# Troisième modélisation : GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(random_state = 0)

reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

y_pred = reg.predict(X_test)

print("Score train :",reg.score(X_train,y_train))

print("Score test :",reg.score(X_test,y_test))

# Afficher représentation de pred test

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10))

plt.scatter(y_pred, y_test, c="green")

plt.plot((y_test.min(),y_test.max()),(y_test.min(),y_test.max()), color = "red")

plt.xlabel("Prédiction")
plt.ylabel("Vraie valeur")

plt.title("Gradient Boosting Regressor pour la prédiction de la consommation")

plt.show();

## Calcul des métriques en comparant 3 modèles, Random Forest, GradientBoosting Regressor et Decision Tree

In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

rf.fit(X_train, y_train)

print("Score train:",rf.score(X_train,y_train))

print("Score test:", rf.score(X_test, y_test))

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


pred_rf = rf.predict(X_test)

pred_train_rf = rf.predict(X_train)

mae_rf_train = mean_absolute_error(y_train, pred_train_rf)

mse_rf_train = mean_squared_error(y_train, pred_train_rf)

rmse_rf_train = mean_squared_error(y_train, pred_train_rf, squared = False)

mae_rf_test = mean_absolute_error(y_test, pred_rf)

mse_rf_test = mean_squared_error(y_test, pred_rf)

rmse_rf_test = mean_squared_error(y_test, pred_rf, squared = False)

In [ ]:
# Calcul des 3 métriques

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

reg_dt = DecisionTreeRegressor()

reg_dt.fit(X_train, y_train)

pred_dt = reg_dt.predict(X_test)

pred_train_dt = reg_dt.predict(X_train)

mae_dt_train = mean_absolute_error(y_train, pred_train_dt)

mse_dt_train = mean_squared_error(y_train, pred_train_dt)

rmse_dt_train = mean_squared_error(y_train, pred_train_dt, squared = False)

mae_dt_test = mean_absolute_error(y_test, pred_dt)

mse_dt_test = mean_squared_error(y_test, pred_dt)

rmse_dt_test = mean_squared_error(y_test, pred_dt, squared = False)

In [ ]:
print("Score train:",reg_dt.score(X_train,y_train))

print("Score test:", reg_dt.score(X_test, y_test))

In [ ]:
# Calcul des métriques de GBoostR :

reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

pred_train_reg = reg.predict(X_train)

mae_reg_train = mean_absolute_error(y_train, pred_train_reg)

mse_reg_train = mean_squared_error(y_train, pred_train_reg)

rmse_reg_train = mean_squared_error(y_train, pred_train_reg, squared = False)

mae_reg_test = mean_absolute_error(y_test, y_pred)

mse_reg_test = mean_squared_error(y_test, y_pred)

rmse_reg_test = mean_squared_error(y_test, y_pred, squared = False)

In [ ]:
# Création d'un df des métriques

data = {'MAE train':[mae_reg_train,mae_dt_train,mae_rf_train],
       'MAE test' :[mae_reg_test,mae_dt_test,mae_rf_test],
       'MSE train' :[mse_reg_train,mse_dt_train,mse_rf_train],
       'MSE test' :[mse_reg_test,mse_dt_test,mse_rf_test],
       'RMSE train':[rmse_reg_train,rmse_dt_train,rmse_rf_train],
       'RMSE test':[rmse_reg_test,rmse_dt_test,rmse_rf_test]}

df = pd.DataFrame(data, index = ['GradientBoostRegression','DecisionTree','RandomForest'])

df.head()

In [ ]:
# # Dernière modélisation avec les variables explicatives : defaut_energie_moy_jour, mois_cos, TMoy, TMax, 
#échanges physiques

target = dfe["consommation"]

feats = dfe[["defaut_energie_moy_jour","mois_cos","TMax","TMoy","ech.physiques" ]]

#Séparation du jeu de données en jeu d'entraînement et jeu de test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size = 0.2)

col = ["TMax","TMoy"]

col_train = X_train[col]

col_test = X_test[col]

# Remplacement des NANs des colonnes température par leurs médianes respectives
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = "median")

X_train.loc[:,col] = imputer.fit_transform(col_train)

X_test.loc[:,col] = imputer.transform(col_test)

#Remplacement des NaN par 0 dans la colonne "defaut énergie moy jour"

X_train.defaut_energie_moy_jour = X_train.defaut_energie_moy_jour.fillna(0)

X_test.defaut_energie_moy_jour = X_test.defaut_energie_moy_jour.fillna(0)


# Standardisation des variables numériques soit colonnes 1 à 3

col_num_train = X_train.iloc[:,1:]

col_num_test = X_test.iloc[:,1:]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

col_num_train = scaler.fit_transform(col_num_train)

col_num_test = scaler.fit(col_num_test)


# Troisième modélisation : GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(random_state = 0)

reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

y_pred = reg.predict(X_test)

print("Score train :",reg.score(X_train,y_train))

print("Score test :",reg.score(X_test,y_test))

# Afficher représentation de pred test

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10))

plt.scatter(y_pred, y_test, c="green")

plt.plot((y_test.min(),y_test.max()),(y_test.min(),y_test.max()), color = "red")

plt.xlabel("Prédiction")
plt.ylabel("Vraie valeur")

plt.title("Gradient Boosting Regressor pour la prédiction de la consommation")

plt.show();

In [ ]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()

rf.fit(X_train, y_train)

print("Score train:",rf.score(X_train,y_train))

print("Score test:", rf.score(X_test, y_test))


from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


pred_rf = rf.predict(X_test)

pred_train_rf = rf.predict(X_train)

mae_rf_train = mean_absolute_error(y_train, pred_train_rf)

mse_rf_train = mean_squared_error(y_train, pred_train_rf)

rmse_rf_train = mean_squared_error(y_train, pred_train_rf, squared = False)

mae_rf_test = mean_absolute_error(y_test, pred_rf)

mse_rf_test = mean_squared_error(y_test, pred_rf)

rmse_rf_test = mean_squared_error(y_test, pred_rf, squared = False)

# Calcul des 3 métriques

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

reg_dt = DecisionTreeRegressor()

reg_dt.fit(X_train, y_train)

pred_dt = reg_dt.predict(X_test)

pred_train_dt = reg_dt.predict(X_train)

mae_dt_train = mean_absolute_error(y_train, pred_train_dt)

mse_dt_train = mean_squared_error(y_train, pred_train_dt)

rmse_dt_train = mean_squared_error(y_train, pred_train_dt, squared = False)

mae_dt_test = mean_absolute_error(y_test, pred_dt)

mse_dt_test = mean_squared_error(y_test, pred_dt)

rmse_dt_test = mean_squared_error(y_test, pred_dt, squared = False)

# Affichage des scores du modèle
print("Score train:",reg_dt.score(X_train,y_train))

print("Score test:", reg_dt.score(X_test, y_test))


# Calcul des métriques de GBoostR :

reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

pred_train_reg = reg.predict(X_train)

mae_reg_train = mean_absolute_error(y_train, pred_train_reg)

mse_reg_train = mean_squared_error(y_train, pred_train_reg)

rmse_reg_train = mean_squared_error(y_train, pred_train_reg, squared = False)

mae_reg_test = mean_absolute_error(y_test, y_pred)

mse_reg_test = mean_squared_error(y_test, y_pred)

rmse_reg_test = mean_squared_error(y_test, y_pred, squared = False)


# Création d'un df des métriques

data = {'MAE train':[mae_reg_train,mae_dt_train,mae_rf_train],
       'MAE test' :[mae_reg_test,mae_dt_test,mae_rf_test],
       'MSE train' :[mse_reg_train,mse_dt_train,mse_rf_train],
       'MSE test' :[mse_reg_test,mse_dt_test,mse_rf_test],
       'RMSE train':[rmse_reg_train,rmse_dt_train,rmse_rf_train],
       'RMSE test':[rmse_reg_test,rmse_dt_test,rmse_rf_test]}

df = pd.DataFrame(data, index = ['GradientBoostRegression','DecisionTree','RandomForest'])

df.head()

## Modélisations Ridge et Lasso

In [ ]:
# Ridge 

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
target = dfe["consommation"]

feats = dfe[["datetime","annee","num_mois","saison","mois_sin","mois_cos","jour_sin","jour_cos","TMin","TMax","TMoy",
             "defaut_energie_moy_jour","prix_base_moyen_ttc","prix_HC_HP_moy_ttc","jour_off"]]

#Séparation du jeu de données en jeu d'entraînement et jeu de test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size = 0.2)

col = ["TMin","TMax","TMoy"]

col_train = X_train[col]

col_test = X_test[col]

# Remplacement des NANs des colonnes température par leurs médianes respectives
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = "median")

X_train.loc[:,col] = imputer.fit_transform(col_train)

X_test.loc[:,col] = imputer.transform(col_test)

#Remplacement des NaN par 0 dans la colonne "defaut énergie moy jour"

X_train.defaut_energie_moy_jour = X_train.defaut_energie_moy_jour.fillna(0)

X_test.defaut_energie_moy_jour = X_test.defaut_energie_moy_jour.fillna(0)

# Remplacement des NANs des colonnes température par leurs médianes respectives
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values = np.nan, strategy = "median")

X_train.loc[:,col] = imputer.fit_transform(col_train)

X_test.loc[:,col] = imputer.transform(col_test)

#Remplacement des NaN par 0 dans la colonne "defaut énergie moy jour"

X_train.defaut_energie_moy_jour = X_train.defaut_energie_moy_jour.fillna(0)

X_test.defaut_energie_moy_jour = X_test.defaut_energie_moy_jour.fillna(0)


# Standardisation des variables numériques 

col_num_train = X_train.iloc[:,1:]

col_num_test = X_test.iloc[:,1:]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

col_num_train = scaler.fit_transform(col_num_train)

col_num_test = scaler.fit(col_num_test)


# Encodage de la variable datetime
import datetime as dt
X_train['datetime']=X_train['datetime'].map(dt.datetime.toordinal)

X_test['datetime']=X_test['datetime'].map(dt.datetime.toordinal)

In [ ]:
from sklearn.linear_model import RidgeCV

ridge_reg = RidgeCV(alphas= (0.001, 0.01, 0.1, 0.3, 0.7, 1, 10, 50, 100))
ridge_reg.fit(X_train, y_train) 

In [ ]:
print('alpha sélectionné par c-v :', ridge_reg.alpha_)
print('score train :', ridge_reg.score(X_train, y_train))
print('score test :', ridge_reg.score(X_test, y_test))

In [ ]:
ridge_pred_train = ridge_reg.predict(X_train)
ridge_pred_test = ridge_reg.predict(X_test)

print('mse train :', mean_squared_error(ridge_pred_train, y_train))
print('mse test :', mean_squared_error(ridge_pred_test, y_test))

In [ ]:
#Lasso
from sklearn.linear_model import Lasso

lasso_r = Lasso(alpha=1)

lasso_r.fit(X_train, y_train)

In [ ]:
lasso_r.coef_

In [ ]:
lasso_reg = Lasso(alpha=0.1)

lasso_reg.fit(X_train, y_train)


In [ ]:
lasso_coef = lasso_reg.coef_

plt.plot(range(len(feats.columns)), lasso_coef)
plt.xticks(range(len(feats.columns)), feats.columns.values, rotation=70);

In [ ]:
print('score train :', lasso_reg.score(X_train, y_train))
print('score test :', lasso_reg.score(X_test, y_test))

In [ ]:
lasso_pred_train = lasso_reg.predict(X_train)
lasso_pred_test = lasso_reg.predict(X_test)

print('mse train :', mean_squared_error(lasso_pred_train, y_train))
print('mse test :', mean_squared_error(lasso_pred_test, y_test))

In [ ]:
from sklearn.linear_model import lasso_path

mes_alphas = (0.001, 0.01, 0.02, 0.025, 0.05, 0.1, 0.25, 0.5, 0.8, 1.0)

alpha_path, coefs_lasso, _ = lasso_path(X_train, y_train, alphas=mes_alphas)

coefs_lasso.shape

In [ ]:
plt.figure(figsize=(10, 7))

for i in range(coefs_lasso.shape[0]):
    plt.plot(alpha_path, coefs_lasso[i,:], '--')

plt.xlabel('Alpha')
plt.ylabel('Coefficients')
plt.title('Lasso path')
plt.legend();


In [ ]:
from sklearn.linear_model import LassoCV

model_lasso = LassoCV(cv=10).fit(X_train, y_train)

alphas = model_lasso.alphas_

plt.figure(figsize=(10, 8))

plt.plot(alphas, model_lasso.mse_path_, ':')
plt.plot(alphas, model_lasso.mse_path_.mean(axis=1), 'k', label='Moyenne', linewidth=2)

plt.axvline(model_lasso.alpha_, linestyle='--', color='k', label='alpha : estimation CV')

plt.xlabel('Alpha')
plt.ylabel('Mean square error')
plt.title('Mean square error pour chaque échantillon')
plt.legend();

In [ ]:
pred_test = model_lasso.predict(X_test)

print('score test :', model_lasso.score(X_test, y_test))
print('mse test :', mean_squared_error(pred_test, y_test))

## Fin 